In [1]:
# Libraries
import pandas as pd
import numpy as np

# Plots
import networkx as nx
import matplotlib
import matplotlib.cm as cm
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots

# Dash
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# 1. Log parsing

In [2]:
path = r'Data\New_PIT2.txt'

#### 1.1 Extract df from text

In [3]:
def txt_to_df(path):

    # 1.1 Read Data
    f = open(path, 'r')
    content = f.read()
    content_list = content.split('/* JOBSPLIT: ')
    f.close()
    df = pd.DataFrame(content_list, columns=['Text'])

    # 1.2. Define Task type
    df['Task Type'] = df["Text"].str.split().str[0]

    # 1.3 Assign Task ID
    df = df.reset_index(drop=True)
    task_list = [0] 
    for index, row in df.iterrows():
        if (row['Task Type'] == 'TASKSTARTTIME') | (row['Task Type'] == 'JOBENDTIME'):
            task_list.append(task_list[index] +1)
        else:
            task_list.append(task_list[index])   
    df['Task ID'] = task_list[1:]

    # 1.4 Remove unessesary text
    df['Text'] = df[['Text','Task Type']].apply(lambda x: x[0].replace('\n\n','\n').replace('STEP SOURCE FOLLOWS */\n','') if x[1]=='STEP' 
                                                          else x[0].replace('*/\n',''), axis = 1)

    return df

df_log = txt_to_df(path)
df_log

,Text,Task Type,Task ID
0,,NaN,0
1,JOBSTARTTIME 27JUL2021:10:34:25.72,JOBSTARTTIME,0
2,TASKSTARTTIME 27JUL2021:10:34:25.72,TASKSTARTTIME,1
3,CATALOG INPUT WORK.SASMAC1.MTF_IFRS9_PD_PIT_BA...,CATALOG,1
4,LIBNAME WORK V9 '/opt/sas/saswork/SAS_work2019...,LIBNAME,1
...,...,...,...
3976,ELAPSED 8,ELAPSED,163
3977,PROCNAME DATASETS,PROCNAME,163
3978,proc datasets lib = work nolist noprint memty...,STEP,163
3979,JOBENDTIME 27JUL2021:10:36:37.94,JOBENDTIME,164


#### 1.2. Extract info about Code

In [4]:
def df_to_code(df):

    #-------------------------------------------
    # 2.1 Extract the Code Syntax
    df_code = df.loc[(df_log['Task Type'].isin(['STEP']))].copy()
    df_code.drop(['Task Type'], inplace=True, axis=1)
    df_code = df_code.rename(columns={'Text': 'Code'})
    df_code = df_code[['Task ID', 'Code']]

    #-------------------------------------------
    # 2.2. Prepare info about Start Time
    df_taskstarttime = df_log.loc[(df_log['Task Type'].isin(['TASKSTARTTIME']))].copy()
    df_taskstarttime['Start Time'] = df_taskstarttime['Text'].str.split().str[1]
    df_taskstarttime['Start Time'] = pd.to_datetime(df_taskstarttime['Start Time'], format='%d%b%Y:%H:%M:%S.%f')
    df_taskstarttime.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.3. Prepare info about Elapsed Time
    df_elapsedtime = df_log.loc[(df_log['Task Type'].isin(['ELAPSED']))].copy()
    df_elapsedtime['Elapsed Time'] = df_elapsedtime['Text'].str.split().str[1].astype('int')/100
    df_elapsedtime.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.4. Prepare info about Procedure Names
    df_procedure = df_log.loc[(df_log['Task Type'].isin(['PROCNAME']))].copy()
    df_procedure['Procedure'] = df_procedure['Text'].str.split().str[1]
    df_procedure.drop(['Text', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 2.5 Output
    df_code = df_code.merge(df_procedure, on=['Task ID'], how = 'outer')\
                     .merge(df_taskstarttime, on=['Task ID'], how = 'outer')\
                     .merge(df_elapsedtime, on=['Task ID'], how = 'outer')
    
    return df_code

df_code = df_to_code(df_log)
df_code

,Task ID,Code,Procedure,Start Time,Elapsed Time
0,1,/*--------------------------------------------...,DATASTEP,2021-07-27 10:34:25.720,0.06
1,2,data _null_;\n set __mtf_pd_pit_version;\n mt...,DATASTEP,2021-07-27 10:34:25.730,0.07
2,3,proc sql noprint;\n create table __mtf_pd_pit...,SQL,2021-07-27 10:34:25.740,29.41
3,4,proc sql noprint;\n create table __mtf_pd_pit...,SQL,2021-07-27 10:34:28.680,24.90
4,5,proc sql noprint;\n select count(*) into :__i...,SQL,2021-07-27 10:34:31.170,1.28
...,...,...,...,...,...
158,159,proc transpose data=WORK.__MTF_PD_PIT_ODR_WID...,TRANSPOSE,2021-07-27 10:36:36.550,6.59
159,160,proc sql noprint _method;\n create table work...,SQL,2021-07-27 10:36:37.210,2.08
160,161,proc datasets lib = work nolist noprint memty...,DATASETS,2021-07-27 10:36:37.420,4.68
161,162,proc datasets lib = work nolist noprint memty...,DATASETS,2021-07-27 10:36:37.890,0.44


#### 1.3. Extract Edges

In [5]:
def df_to_edges(df):
    
    #-------------------------------------------
    # 3.1 Extract Table info
    # 3.1.a Get data
    df_tables = df.loc[(df['Task Type'].isin(['DATASET', 'OPENTIME', 'TASKSTARTTIME',  'JOBENDTIME']))].copy()   

    # 3.1.b # Extract table name and type
    df_tables['Table'] = np.where(df_tables['Task Type'].isin(['DATASET']),
                                  df_tables['Text'].str.split().str[3],
                                  np.where(df_tables['Task Type'].isin(['OPENTIME']),
                                           df_tables['Text'].str.split().str[1],
                                           ''))
    df_tables['Table type'] = df_tables['Table'].str.split('.').str[-1]                          # Extract table type
    df_tables['Table'] = df_tables['Table'].apply(lambda x: '.'.join(x.split('.')[:-1]))         # Extract table name

    # 3.1.c Define table type (Input, Output, Update)
    df_tables['Dataset type'] = np.where(df_tables['Task Type'].isin(['DATASET']),
                                          df_tables['Text'].str.split().str[1],                  # Define Input, Output, Update tables
                                          '')
    df_tables['Dataset type'] = np.where(df_tables['Task Type'].isin(['OPENTIME']),
                                          df_tables['Dataset type'].shift(),                     # Inherit table type
                                          df_tables['Dataset type'])

    # 3.1.d Add sub-step indicator
    df_tables = df_tables.reset_index(drop=True)
    df_tables['Lag Task ID'] = df_tables['Task ID'].shift()
    df_tables['Lag Dataset type'] = df_tables['Dataset type'].shift()
    sub_step_list = [1]
    for index, row in df_tables.iterrows():
        if (row['Task ID'] != row['Lag Task ID']):
            sub_step_list.append(1)                       # Reset sub-step
        elif ((row['Lag Dataset type'] == 'OUTPUT') & (row['Dataset type'] == 'INPUT')) | \
             ((row['Lag Dataset type'] == 'OUTPUT') & (row['Dataset type'] == 'UPDATE')) | \
             ((row['Lag Dataset type'] == 'UPDATE') & (row['Dataset type'] == 'INPUT')):
            sub_step_list.append(sub_step_list[index] +1) # Increase sub-step
        else:
            sub_step_list.append(sub_step_list[index])    # Keep sub-step
    df_tables['SubTask ID'] = sub_step_list[1:]

    # 3.1.e Correct case for updated tables
    df_tables['Dataset type'] = df_tables['Dataset type'].replace({'UPDATE': 'UPDATE INPUT'})
    df_update = df_tables.loc[df_tables['Dataset type'].isin(['UPDATE INPUT'])].copy()
    df_update['Dataset type'] = df_update['Dataset type'].replace({'UPDATE INPUT': 'UPDATE OUTPUT'})
    df_tables = df_tables.append(df_update).sort_values(['Task ID', 'SubTask ID', 'Dataset type'])
    
    # 3.1.f Add node index
    node_id_dict = {}
    node_id_list = []
    for index, row in df_tables.iterrows():
        # If update or output:
        if (row['Dataset type'] == 'OUTPUT') | (row['Dataset type'] == 'UPDATE OUTPUT') :
            # create node ID
            node_id = str(row['Task ID']) +':'+ \
                      str(row['SubTask ID']) +':'+ \
                      row['Table'] +':'+ \
                      row['Table type']   
            # update column
            node_id_list.append(node_id)       
            # update dict
            node_id_dict[row['Table']] = node_id                                                                    

        # If source table:
        elif row['Table'] not in node_id_dict:
            # create node ID
            node_id = '0:0:' + \
                      row['Table']  +':'+ \
                      row['Table type']
            # update column
            node_id_list.append(node_id)                                                           
            # update dict
            node_id_dict[row['Table']] = node_id  
        # If input:
        else: 
            # use previous node ID 
            node_id_list.append(node_id_dict[row['Table']])                                                         
    df_tables['Node Id'] = node_id_list

    # 3.1.g Remove leftover tables
    df_tables.drop(['Table type','Lag Task ID','Lag Dataset type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.2. Prepare info about Input tables
    df_input = df_tables.loc[(df_tables['Task Type'].isin(['DATASET']) ) & 
                             (df_tables['Dataset type'].isin(['INPUT', 'UPDATE INPUT']))].copy()
    df_input = df_input.rename(columns={'Table': 'Input Table',
                                        'Node Id': 'Source ID'})
    df_input.drop(['Text', 'Dataset type', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.3. Prepare info about Output tables
    df_output = df_tables.loc[(df_tables['Task Type'].isin(['DATASET']) ) & 
                              (df_tables['Dataset type'].isin(['OUTPUT', 'UPDATE OUTPUT']))].copy()
    df_output = df_output.rename(columns={'Table': 'Output Table',
                                          'Node Id': 'Target ID'})
    df_output.drop(['Text', 'Dataset type', 'Task Type'], inplace=True, axis=1)

    #-------------------------------------------
    # 3.4. Prepare info about Time Calculations
    # 3.4.a Get Data
    df_time = df_tables.loc[df_tables['Task Type'].isin(['OPENTIME', 'TASKSTARTTIME', 'JOBENDTIME'])].copy()

    # 3.4.b Add time
    df_time['Start Time'] = np.where(df_time['Task Type'].isin(['OPENTIME']),
                                     df_time['Text'].str.split().str[2].str.replace('DATE:', ''),
                                     df_time['Text'].str.split().str[1].str.replace('DATE:', ''))
    df_time['Start Time']  = pd.to_datetime(df_time['Start Time'], format='%d%b%Y:%H:%M:%S.%f')

    # 3.4.c Remove columns and rows
    df_time.drop(['Table','Text','Task Type','Dataset type','Node Id'], inplace=True, axis=1)
    df_time = df_time.drop_duplicates(subset=['Task ID', 'SubTask ID'], keep='first')

    # 3.4.d Calculate time for each step
    df_time['Elapsed Time'] = (df_time['Start Time'].shift(-1) - df_time['Start Time']).dt.total_seconds()
    df_time['Elapsed Time'] = df_time['Elapsed Time'].round(2)

    #-------------------------------------------
    # 3.5. Output
    # 3.5.a Merge the results for input and output tables + Elapsed time
    df = df_input.merge(df_output, on=['Task ID', 'SubTask ID'], how = 'outer')\
                 .merge(df_time, on=['Task ID', 'SubTask ID'], how = 'left')\
                 .sort_values(['Task ID', 'SubTask ID'])\
                 .reset_index(drop = True)

    # 3.5.b Fill missing for specific cases
    df['Input Table'] = df['Input Table'].fillna('No Input')
    df['Output Table'] = df['Output Table'].fillna('_null_')

    # 3.5.c Correct Start time
    df['Start Time'] = df['Start Time'].fillna(df['Start Time'].shift()) 
    df['Elapsed Time'] = df['Elapsed Time'].fillna(0)

    # 3.5.d Correct Target node ID
    df['Target ID'] = df['Target ID'].fillna(df['Task ID'].astype('str') +':'+ df['SubTask ID'].astype('str')+ ':No Output:Empty') 

    # 3.5.e Correct Source node ID
    df['Source ID'] = df[['Input Table','Task ID','SubTask ID','Source ID']].apply(lambda x: str(x[1])+':'+str(int(x[2])-1)+':No Input:Empty' if x[0]=='No Input' 
                                                                                  else x[3], axis = 1)
    
    # 3.5.f Reorder columns
    df = df[['Task ID','SubTask ID', 'Source ID','Target ID', 'Input Table','Output Table', 'Start Time','Elapsed Time']]
    
    return df

df_edges = df_to_edges(df_log)
df_edges

,Task ID,SubTask ID,Source ID,Target ID,Input Table,Output Table,Start Time,Elapsed Time
0,1,1,1:0:No Input:Empty,1:1:WORK.__MTF_PD_PIT_VERSION:DATA,No Input,WORK.__MTF_PD_PIT_VERSION,2021-07-27 10:34:25.720,0.01
1,2,1,1:1:WORK.__MTF_PD_PIT_VERSION:DATA,2:1:No Output:Empty,WORK.__MTF_PD_PIT_VERSION,_null_,2021-07-27 10:34:25.730,0.01
2,3,1,0:0:INPUT.PD_VIEW:DATA,3:1:WORK.__MTF_PD_PIT_DISTINCT_RATING1:DATA,INPUT.PD_VIEW,WORK.__MTF_PD_PIT_DISTINCT_RATING1,2021-07-27 10:34:25.740,2.93
3,3,1,0:0:INPUT.PD_VIEW:DATA,3:1:WORK.'SASTMP-000000348':UTILITY,INPUT.PD_VIEW,WORK.'SASTMP-000000348',2021-07-27 10:34:25.740,2.93
4,3,2,0:0:INPUT.PD_RR_RATING:DATA,3:2:WORK.__MTF_PD_PIT_DISTINCT_RATING2:DATA,INPUT.PD_RR_RATING,WORK.__MTF_PD_PIT_DISTINCT_RATING2,2021-07-27 10:34:28.670,0.00
...,...,...,...,...,...,...,...,...
286,160,1,159:1:WORK.__MTF_PD_PIT_ODR_LONG_DIS:DATA,160:1:WORK.TEST_PD_DIS_IN:DATA,WORK.__MTF_PD_PIT_ODR_LONG_DIS,WORK.TEST_PD_DIS_IN,2021-07-27 10:36:37.210,0.21
287,160,1,100:1:WORK.__MTF_PD_PIT_MARGINAL_BOTH_PD:DATA,160:1:WORK.TEST_PD_DIS_IN:DATA,WORK.__MTF_PD_PIT_MARGINAL_BOTH_PD,WORK.TEST_PD_DIS_IN,2021-07-27 10:36:37.210,0.21
288,161,1,161:0:No Input:Empty,161:1:WORK.'SASTMP-000000636':UTILITY,No Input,WORK.'SASTMP-000000636',2021-07-27 10:36:37.420,0.47
289,162,1,162:0:No Input:Empty,162:1:WORK.'SASTMP-000000638':UTILITY,No Input,WORK.'SASTMP-000000638',2021-07-27 10:36:37.890,0.04


## 2. Network plot

#### 2.1. Create Graph

In [6]:
G = nx.from_pandas_edgelist(df_edges,
                            'Source ID','Target ID',
                            ['Task ID','SubTask ID',
                             'Input Table','Output Table',
                             'Start Time','Elapsed Time'],
                            create_using=nx.DiGraph())

#### 2.2 Define positions

In [7]:
def get_coords(G):
    # get a list of all nodes 
    list_nodes_full = sorted(list(G.nodes))

    def setdiff_sorted(list_append, list_result):
        ans = np.setdiff1d(list_append, list_result, False).tolist()
        list_result.extend(ans)
        return ans

    # get source nodes 
    source_nodes = sorted([node for node in G.nodes() if G.in_degree(node) == 0])

    list_nodes = []
    for i in range(len(source_nodes)):

        # Define source node
        source = source_nodes[i]

        # Get nodes in depth order
        list_nodes_depth = list(nx.dfs_preorder_nodes(G, source=source, depth_limit=None))

        # Keep nodes not already in list
        list_nodes_unique = setdiff_sorted(list_nodes_depth, list_nodes)

    # Get missed nodes
    list_nodes_miss = setdiff_sorted(list_nodes, list_nodes_full)

    # Create df
    df_nodes = pd.DataFrame(list_nodes, columns =['Node'])
    df_nodes['Task ID'] = df_nodes['Node'].str.split(':').str[0].astype('int')
    df_nodes['SubTask ID'] = df_nodes['Node'].str.split(':').str[1].astype('int')

    # Assign x
    df_nodes['x'] = df_nodes['Task ID']

    # Assign y
    df_nodes = df_nodes.reset_index(drop=True)
    df_nodes['Lag Task ID'] = df_nodes['Task ID'].shift()
    df_nodes['Lag SubTask ID'] = df_nodes['SubTask ID'].shift()
    x_list = [10] 
    for index, row in df_nodes.iterrows():
        if   ( (row['Task ID']-row['Lag Task ID'])==0  & (row['SubTask ID'] != row['Lag SubTask ID']) ):
            x_list.append(x_list[index] + 1)
        elif ( (row['Task ID']-row['Lag Task ID'])<= 0 ):
            x_list.append(x_list[index] + 1)        
        else:
            x_list.append(x_list[index]) 
    df_nodes['y'] = x_list[1:]

    # Coords dict
    coords = df_nodes[['Node', 'x' , 'y']].set_index('Node').T.to_dict('list')
    return coords


# Add coords
coords = get_coords(G)
# Add attributes
nx.set_node_attributes(G, coords, 'coords') 


#### 2.3. Create Edges

In [8]:
def get_edge_trace(G):
    #====================================================================================        
    # Edge info
    edge_x = []
    edge_y = []
    edge_text = []
    edge_x_text = []
    edge_y_text = []
    edge_color_values = []
    for edge in G.edges():
        #--------------------------------------------------------------------
        # Coords
        x0, y0 = G.nodes[edge[0]]['coords']
        x1, y1 = G.nodes[edge[1]]['coords']
        x_mean = round( (x0+x1)/2, 4)
        y_mean = round( (y0+y1)/2, 4)   
        edge_x.append([x0, x_mean, x1, None])
        edge_y.append([y0, y_mean, y1, None])   
        
        #--------------------------------------------------------------------
        # Hover text
        edge_text.append('Task ID: '+str(G.edges[edge[0],edge[1]]['Task ID']) 
                         +' '+
                         'SubTask ID: '+str(G.edges[edge[0],edge[1]]['SubTask ID'])
                         +'<br> '+
                         'Input: '+str(G.edges[edge[0],edge[1]]['Input Table'])
                         +'<br> '+   
                         'Output: '+str(G.edges[edge[0],edge[1]]['Output Table'])     
                         +'<br> '+                     
                         'Elapsed Time: '+str(G.edges[edge[0],edge[1]]['Elapsed Time']) 
                        )
        edge_x_text.append(x_mean)
        edge_y_text.append(y_mean) 
        
        #--------------------------------------------------------------------
        # Colorbar
        edge_color_values.append(G.edges[edge[0],edge[1]]['Elapsed Time'])
        
    #====================================================================================        
    # b) Edges color
    minima = min(edge_color_values)
    maxima = max(edge_color_values)
    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap='RdYlGn_r')
    edge_color = []
    for v in edge_color_values:
        rgba = mapper.to_rgba(v)
        edge_color.append( matplotlib.colors.to_hex(rgba, keep_alpha=False))
        
    #====================================================================================        
    # c) Edges lines
    edge_trace = []  
    for i in range(len(edge_color)):
        edge_trace.append(go.Scatter(x=edge_x[i], y=edge_y[i],
                                     mode = 'lines', 
                                     line_shape = 'spline',
                                     line = dict(width=1, 
                                                 dash='dot', 
                                                 color=edge_color[i]),
                                     hoverinfo = 'none',
                                     showlegend=False))
        
    #====================================================================================        
    # Text
    edge_text_trace = go.Scatter(x=edge_x_text, y=edge_y_text, 
                                 # Marker
                                 mode = 'markers', 
                                 marker_symbol = 'hexagram',
                                 marker=dict(showscale=True, 
                                             colorscale='RdYlGn', 
                                             reversescale=True,
                                             size = 8, 
                                             color=edge_color_values,
                                             colorbar=dict(thickness=15,
                                                           title='Execution time (s)',
                                                           xanchor='left',
                                                           titleside='right')
                                            ),
                                 # Text
                                 text = edge_text,  
                                 textposition = 'top center', 
                                 hovertemplate = ' %{text}',
                                 showlegend=False)
    
    return edge_trace, edge_text_trace

edge_trace, edge_text_trace = get_edge_trace(G)

#### 2.4 Create Nodes

In [9]:
def get_node_trace(G):
    #====================================================================================            
    # 1. Node Attributes
    node_x = []
    node_y = []
    node_group = []  
    node_label = []
    node_color = []
    node_shape = []
    step_id = []    
    for node in G.nodes():  
        #--------------------------------------------------------------------
        # Node Coords
        x, y = G.nodes[node]['coords']
        node_x.append(x)
        node_y.append(y)        

        #--------------------------------------------------------------------
        # Node name
        table_name = node.split(':')[2]
        node_label.append(table_name)
        
        #--------------------------------------------------------------------
        # Step ID
        step_id.append(node.split(':')[0])    
        
        #--------------------------------------------------------------------
        # Table Type
        table_type = node.split(':')[3]       
        
        #--------------------------------------------------------------------
        # Predesessors
        predecessors = [c.split(':')[2] for c in G.predecessors(node)]

        
        #--------------------------------------------------------------------
        # Node shape and color
        # a) No Input node
        if table_name == 'No Input':
            node_group.append('No Input')
            node_shape.append('diamond-cross')
            node_color.append('grey')
        # b) No Output node
        elif table_name == 'No Output':
            node_group.append('No Output')
            node_shape.append('square-cross')
            node_color.append('grey') 
        # c) Temporary tables
        elif table_type == 'UTILITY':
            node_group.append('Technical Table')            
            node_shape.append('star-square-dot')
            node_color.append('silver')    
        # d) Input node
        elif G.in_degree(node) == 0:
            node_group.append('Input Table')              
            node_shape.append('diamond')
            node_color.append('gold')
        # e) Output node
        elif G.out_degree(node) == 0:
            node_group.append('Output Table')                
            node_shape.append('square')
            node_color.append('cyan') 
        # f) Updated node            
        elif table_name in predecessors:
            node_group.append('Updated Table')             
            node_shape.append('cross')
            node_color.append('orange')
        # g) Internal nodes
        else:
            node_group.append('Internal Table')                   
            node_shape.append('circle')
            node_color.append('blue')

    #====================================================================================        
    # 2. Node plot
    node_trace = []
    for elements in ['No Input', 'No Output', 'Technical Table', 'Input Table', 'Output Table', 'Updated Table', 'Internal Table']:
        # Get indices of nodes
        indices = [i for i, j in enumerate(node_group) if j == elements]
        node_x_group = [node_x[i] for i in indices]
        node_y_group = [node_y[i] for i in indices]
        node_label_group = [node_label[i] for i in indices]
        node_color_group = [node_color[i] for i in indices]
        node_shape_group = [node_shape[i] for i in indices]
        step_id_group = [step_id[i] for i in indices]
        
        node_trace.append(go.Scatter(x = node_x_group, 
                                     y = node_y_group,
                                     mode = 'markers',
                                     marker=dict(symbol = node_shape_group, 
                                                 color = node_color_group,
                                                 size = 10
                                                ),
                                     name = elements,
                                     marker_line_color='black', 
                                     marker_line_width=0.5,
                                     text = node_label_group,
                                     textposition = 'top center',
                                     hovertemplate = ' %{text}',
                                     meta = step_id_group,
                                     showlegend=True)
                        ) 
    return node_trace


node_trace = get_node_trace(G)

## 3. Dash plot

In [10]:
# Server start
app = dash.Dash()

# network plot
fig_net = go.Figure(edge_trace + node_trace+[edge_text_trace]) 
fig_net.update_layout(xaxis=dict(showgrid=False, 
                                 zeroline=False),
                      yaxis=dict(showgrid=False, 
                                 zeroline=False),
                     legend=dict(orientation="h",
                                 x=1, y=1.02,
                                 xanchor="right", yanchor="bottom")
                     )

# Dash plot
app.layout = html.Div([
    # Network plot
    dcc.Graph(id='basic_graph',
              figure=fig_net),
    # Table base
    html.Div(className='row', 
             children=[
                       # Table
                       dash_table.DataTable(id='table',
                                            columns=[{"name": i, "id": i} for i in df_code.columns],
                                            data=df_code.to_dict('records'),
                                            fixed_rows={'headers': True},
                                            style_table={'height': 450,
                                                         'overflowY': 'scroll',
                                                         'border': 'thin lightgrey solid'},  
                                            style_cell={'textAlign': 'left',
                                                        'whiteSpace': 'pre-line',
                                                        'backgroundColor': 'rgb(153, 204, 255)',
                                                        'color': 'black',
                                                        'minWidth': '10px', 
                                                        'maxWidth': '800px'},
                                            style_header={'fontWeight': 'bold',
                                                          'backgroundColor': 'rgb(0, 0, 153)',
                                                          'color': 'white'}
                                           )
             ])
])

# https://community.plotly.com/t/update-a-dash-datatable-with-callbacks/21382/3
# https://dash.plotly.com/datatable/editable

# Add interactive table
@app.callback(
    Output('table', 'data'),
    Input('basic_graph', 'selectedData'))
def display_relayout_data(selectedData):
    if selectedData is not None:
        x_values = []
        for elements in selectedData['points']:
             if 'meta' in elements:
                if elements['meta'] not in x_values:
                    x_values.append( int(elements['meta']) )
        df = df_code.loc[ df_code['Task ID'].isin(x_values) ]
    else:
        df = df_code
    return df.to_dict('records')

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Aug/2021 19:49:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:04] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:04] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2021 19:49:45] "POST /_dash-update-component HTTP/1.1" 200 -
